<a href="https://colab.research.google.com/github/yhanyi/MLNotebooks/blob/main/MyAnimeListTopGenres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!pip install chromedriver-autoinstaller
import sys
sys.path.insert(0, "/usr/bin/chromdriver")

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
service = Service(executable_path=r"/usr/bin/chromedriver")
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
links = []
for page in range(0, 1000, 50):
  driver.get("https://myanimelist.net/topanime.php?limit="+str(page))
  link = driver.find_elements(By.CSS_SELECTOR, 'div[class="detail"] h3 a')
  for item in link: links.append(item.get_attribute("href"))

In [ ]:
wait = WebDriverWait(driver, 5)
max_attempts = 5
titles = []
ratings = []
genres = []
for item_link in links:
  driver.get(item_link)
  for _ in range(max_attempts):
    try:
        title = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div[class="h1-title"]')))
        rating = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div[class*="score-label"]')))
        genre = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[contains(text(), "Genre")]/parent::div')))
        titles.append(title.text)
        ratings.append(rating.text)
        genres.append(genre.text)
        break
    except Exception:
      continue

In [ ]:
raw_data = {"Title": titles, "Rating": ratings, "Genre": genres}
raw_df = pd.DataFrame(data=raw_data)

In [ ]:
def splitstr(string):
  return string.replace("Genres: ","").replace("Genre: ","").split(", ")

In [ ]:
genres = []
for i in range(len(raw_df["Genre"])):
  genre = splitstr(raw_df["Genre"][i])
  raw_df["Genre"][i] = genre
  for g in genre:
    if g not in genres: genres.append(g)
genres.sort()

In [ ]:
data = {}
for g in genres:
  data[g] = []
  for i in range(len(raw_df)):
    if g not in raw_df["Genre"][i]:
      data[g].append(0)
    else:
      data[g].append(1)
data["Ratings"] = ratings

In [ ]:
df = pd.DataFrame(data=data)

In [ ]:
df

,Action,Adventure,Avant Garde,Award Winning,Boys Love,Comedy,Drama,Ecchi,Fantasy,Girls Love,Gourmet,Horror,Mystery,Romance,Sci-Fi,Slice of Life,Sports,Supernatural,Suspense,Ratings
0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,9.13
1,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,9.09
2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,9.07
3,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,9.06
4,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,9.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,7.78
980,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,7.78
981,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,7.78
982,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,7.78


## Linear Regression

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [ ]:
train_data, test_data = train_test_split(df, test_size = 0.1)

In [ ]:
X_train, y_train = train_data[train_data.columns[:-1]], train_data[train_data.columns[-1]]
X_test, y_test = test_data[test_data.columns[:-1]], test_data[test_data.columns[-1]]

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(X_train, y_train)

LinearRegression()

In [ ]:
y_pred = lr.predict(X_test)

In [ ]:
print("MSE: ", mean_squared_error(y_test, y_pred))
print("MAE: ", mean_absolute_error(y_test, y_pred))

MSE:  0.08307411640822937
MAE:  0.2341095141830638


In [ ]:
def convert(lst):
  res = {}
  for g in genres:
    res[g] = []
    if g not in lst:
      res[g].append(0)
    else:
      res[g].append(1)
  return pd.DataFrame(res)

In [ ]:
x = len(genres)
top_score = 0
top_com = []
bot_score = 10
bot_com = []
for i in range(0, x-2):
  for j in range(i+1, x-1):
    for k in range(j+1, x):
      lst = [genres[i],genres[j],genres[k]]
      score = lr.predict(convert(lst))
      if score > top_score:
        top_com = lst
        top_score = score
      if score < bot_score:
        bot_com = lst
        bot_score = score
print("Top: ", top_com, top_score)
print("Bottom: ", bot_com, bot_score)

Top:  ['Sports', 'Supernatural', 'Suspense'] [8.47607438]
Bottom:  ['Avant Garde', 'Girls Love', 'Gourmet'] [7.58147889]


In [ ]:
x = len(genres)
top_score = 0
top_com = []
bot_score = 10
bot_com = []
for i in range(0, x-3):
  for j in range(i+1, x-2):
    for k in range(j+1, x-1):
      for l in range(k+1, x):
        lst = [genres[i],genres[j],genres[k],genres[l]]
        score = lr.predict(convert(lst))
        if score > top_score:
          top_com = lst
          top_score = score
        if score < bot_score:
          bot_com = lst
          bot_score = score
print("Top: ", top_com, top_score)
print("Bottom: ", bot_com, bot_score)

Top:  ['Drama', 'Sports', 'Supernatural', 'Suspense'] [8.56379037]
Bottom:  ['Avant Garde', 'Girls Love', 'Gourmet', 'Horror'] [7.52317412]


In [ ]:
x = len(genres)
top_score = 0
top_com = []
bot_score = 10
bot_com = []
for i in range(0, x-4):
  for j in range(i+1, x-3):
    for k in range(j+1, x-2):
      for l in range(k+1, x-1):
        for m in range(l+1, x):
          lst = [genres[i],genres[j],genres[k],genres[l],genres[m]]
          score = lr.predict(convert(lst))
          if score > top_score:
            top_com = lst
            top_score = score
          if score < bot_score:
            bot_com = lst
            bot_score = score
print("Top: ", top_com, top_score)
print("Bottom: ", bot_com, bot_score)

Top:  ['Award Winning', 'Drama', 'Sports', 'Supernatural', 'Suspense'] [8.64856677]
Bottom:  ['Avant Garde', 'Girls Love', 'Gourmet', 'Horror', 'Romance'] [7.47878398]
